This is the data setup for an NFL machine learning model. This file will be for the data ingestion and transformation, and I will have multiple other files for different types of modeling.

# Imports & Housekeeping

In [1]:
# Basic Packages
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime as dt

# Visualizations
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Notebook display options
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)

# Data Import

This is not the full raw data. This data was acquired in it's raw form from nfl_data_py api. Transformations and EPA calculations were done in Pycharm because of easier access to classes, functions, etc. At this point, Google Notebooks is a better place for the modeling. Credit: https://github.com/cooperdff/nfl_data_py.

In [10]:
nfl_api = pd.read_csv("../API Data Out/test_2020_to_2023.csv", index_col=0)
schedule = pd.read_csv("../API Data Out/schedule_2020_to_2023.csv", index_col=0)

In [11]:
# We want a df that we will model on without anything that isn't a feature or target
model_df = nfl_api

model_df.sort_values(by='game_id', ascending=False)

,game_id,season,week,team,opponent,score,home,...,ewma_dynamic_window_rushing_defense_team,ewma_dynamic_window_passing_defense_team,ewma_dynamic_window_rushing_offense_opp,ewma_dynamic_window_passing_offense_opp,ewma_dynamic_window_rushing_defense_opp,ewma_dynamic_window_passing_defense_opp,team_id
1739,2023_02_WAS_DEN,2023,2,WAS,DEN,35,0,...,-0.19,-0.09,-0.03,-0.10,-0.16,0.17,2023_02_WAS_DEN_A
1738,2023_02_WAS_DEN,2023,2,DEN,WAS,33,1,...,-0.16,0.17,-0.10,-0.17,-0.19,-0.09,2023_02_WAS_DEN_H
1737,2023_02_SF_LA,2023,2,SF,LA,30,0,...,-0.07,-0.15,0.02,-0.02,-0.01,-0.02,2023_02_SF_LA_A
1736,2023_02_SF_LA,2023,2,LA,SF,23,1,...,-0.01,-0.02,-0.08,0.17,-0.07,-0.15,2023_02_SF_LA_H
1735,2023_02_SEA_DET,2023,2,SEA,DET,37,0,...,0.01,0.15,-0.12,0.21,0.01,-0.08,2023_02_SEA_DET_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2020_01_CLE_BAL,2020,1,CLE,BAL,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2020_01_CLE_BAL_A
2,2020_01_CHI_DET,2020,1,DET,CHI,23,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2020_01_CHI_DET_H
3,2020_01_CHI_DET,2020,1,CHI,DET,27,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2020_01_CHI_DET_A
1,2020_01_ARI_SF,2020,1,ARI,SF,24,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2020_01_ARI_SF_A


Ok, this is our starting df. The target is score_diff - The difference in score between two teams. In this case, the score diff is with respect to the home team. This mean if the diff is positive, the home team won. We will keep this in mind for bringing in other features.

The EPA columns are created by bringing in play by play data, rolling them up into games, and then creating a 10 game rolling average offset by one week so that each EPA that we see is an average of the last 10 games. If the team has not played 10 games in the season, it will take as many games as have been played.

# (Other) Feature Engineering

Adding in a couple of features I think will be valuable to model on

In [7]:
feature_df = schedule
feature_df.columns

Index(['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
       'gametime', 'away_team', 'away_score', 'home_team', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game',
       'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id',
       'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium'],
      dtype='object')

In [8]:
# Was wind involved in the game. I'm setting the threshold at 15mph, but may adjust later
feature_df['windy'] = np.where(schedule.loc[:, 'wind'] > 15, 1, 0)

# Next lets look at "rest differential". A further analysis of this can be seen on my github
feature_df['rest_differential'] = schedule.loc[:, "home_rest"] - schedule.loc[:, "away_rest"]
feature_df

,game_id,season,game_type,week,gameday,weekday,gametime,...,away_coach,home_coach,referee,stadium_id,stadium,windy,rest_differential
5583,2020_01_HOU_KC,2020,REG,1,2020-09-10,Thursday,20:20,...,Bill O'Brien,Andy Reid,Clete Blakeman,KAN00,Arrowhead Stadium,0,0
5584,2020_01_SEA_ATL,2020,REG,1,2020-09-13,Sunday,13:00,...,Pete Carroll,Dan Quinn,Shawn Hochuli,ATL97,Mercedes-Benz Stadium,0,0
5585,2020_01_CLE_BAL,2020,REG,1,2020-09-13,Sunday,13:00,...,Kevin Stefanski,John Harbaugh,Ronald Torbert,BAL00,M&T Bank Stadium,0,0
5586,2020_01_NYJ_BUF,2020,REG,1,2020-09-13,Sunday,13:00,...,Adam Gase,Sean McDermott,Shawn Smith,BUF00,New Era Field,0,0
5587,2020_01_LV_CAR,2020,REG,1,2020-09-13,Sunday,13:00,...,Jon Gruden,Matt Rhule,Brad Allen,CAR00,Bank of America Stadium,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6688,2023_18_ATL_NO,2023,REG,18,2024-01-07,Sunday,13:00,...,Arthur Smith,Dennis Allen,NaN,NOR00,Mercedes-Benz Superdome,0,-1
6689,2023_18_PHI_NYG,2023,REG,18,2024-01-07,Sunday,13:00,...,Nick Sirianni,Brian Daboll,NaN,NYC01,MetLife Stadium,0,0
6690,2023_18_LA_SF,2023,REG,18,2024-01-07,Sunday,13:00,...,Sean McVay,Kyle Shanahan,NaN,SFO01,Levi's Stadium,0,-7
6691,2023_18_JAX_TEN,2023,REG,18,2024-01-07,Sunday,13:00,...,Doug Pederson,Mike Vrabel,NaN,NAS00,Nissan Stadium,0,0


Ok lets reduce the columns to the ones we need and merge with our original df

In [12]:
merge_df = feature_df[['game_id', 'rest_differential', 'windy', 'div_game', 'spread_line', 'total_line', 'game_type', 'location', 'total']]

In [13]:
final_df = pd.merge(model_df, merge_df, on="game_id", how='left')

# Only reg season games
final_df = final_df.loc[final_df.game_type == "REG"]

In [14]:
# Since we're now doing 1 team per row, need to make some adjustments to the values
final_df["implied_points"] = np.where(final_df.home == 1, final_df.total_line/2 - final_df.spread_line/2, final_df.total_line/2 + final_df.spread_line/2)
final_df.spread_line = np.where(final_df.home == 1, final_df.spread_line, final_df.spread_line*-1)
final_df.rest_differential = np.where(final_df.home == 1, final_df.rest_differential, final_df.rest_differential*-1)

In [21]:
final_df.tail(50)

,game_id,season,week,team,opponent,score,home,...,div_game,spread_line,total_line,game_type,location,total,implied_points
1690,2023_01_HOU_BAL,2023,1,BAL,HOU,25,1,...,0,9.50,43.50,REG,Home,34.00,17.00
1691,2023_01_HOU_BAL,2023,1,HOU,BAL,9,0,...,0,-9.50,43.50,REG,Home,34.00,26.50
1692,2023_01_JAX_IND,2023,1,IND,JAX,21,1,...,1,-4.00,45.50,REG,Home,52.00,24.75
1693,2023_01_JAX_IND,2023,1,JAX,IND,31,0,...,1,4.00,45.50,REG,Home,52.00,20.75
1694,2023_01_LA_SEA,2023,1,SEA,LA,13,1,...,1,4.50,45.00,REG,Home,43.00,20.25
1695,2023_01_LA_SEA,2023,1,LA,SEA,30,0,...,1,-4.50,45.00,REG,Home,43.00,24.75
1696,2023_01_LV_DEN,2023,1,DEN,LV,16,1,...,1,3.00,43.00,REG,Home,33.00,20.00
1697,2023_01_LV_DEN,2023,1,LV,DEN,17,0,...,1,-3.00,43.00,REG,Home,33.00,23.00
1698,2023_01_MIA_LAC,2023,1,LAC,MIA,34,1,...,0,3.00,50.50,REG,Home,70.00,23.75
1699,2023_01_MIA_LAC,2023,1,MIA,LAC,36,0,...,0,-3.00,50.50,REG,Home,70.00,26.75


ok lets output back to csv so that we can bring it in the other files to model on without messing with the final result

In [22]:
final_df.to_csv("Data/final_data.csv")

Okay we should probably do some visualizations to see what this data actually is telling us